In [1]:
# 前根收 ema 38 以下，此根收 ema 38 以上，且量大於前根 2x
# ! conda install -c conda-forge ta --yes

In [2]:
import requests
import pandas as pd
import ta
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

In [3]:
timezone = 8
endpoint = 'wss://stream.binance.com:9443/ws'
symbol = 'ethusdt'
symbol_C = symbol.upper()
interval = '1h'

# start epoch till now
start_time = 1640999048000
end_time = round(time.time() * 1000)
# end_time = 1672556207000

# step between timestamps in milliseconds
step = 60000 * 3600

In [4]:
tp_atr = 18

# stop loss determine candlestick
sl_det = 'Close'

ema_int = 50

In [5]:
def get_historical(symbol, interval, start_time, end_time, step):
    
    raw_df = pd.DataFrame()
    
    url = "https://api.binance.com/api/v3/klines"
    
    for timestamp in range(start_time, end_time, step):
        params = {"symbol": symbol_C,
                  "interval": interval,
                  "startTime": timestamp,
                  "endTime": timestamp + step}
        response = requests.get(url, params=params).json()
        out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                               "Volume", "Close_Time", "Quote asset volume",
                                               "Number of trades", "Taker buy base asset volume",
                                               "Taker buy quote asset volume", "Ignore"])
        raw_df = pd.concat([raw_df, out], axis = 0)
    
    raw_df = raw_df[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]
    convert_dict = {'Close_Time': float, 'Open': float, 'Close': float, "High": float, "Low": float, 'Volume': float}
    raw_df = raw_df.astype(convert_dict)
    
    raw_df['Close_Time'] = pd.to_datetime(raw_df['Close_Time'], unit = 'ms')
    raw_df['Close_Time'] = raw_df['Close_Time'] + pd.Timedelta(hours=timezone)
    raw_df['Close_Time'] = raw_df['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    raw_df = raw_df.reset_index(drop=True)
    
    return raw_df

In [6]:
raw_df = get_historical(symbol, interval, start_time, end_time, step)
print(raw_df)

                Close_Time     Open    Close     High      Low      Volume
0      2022-01-01 10:59:59  3724.88  3728.32  3738.47  3722.23   5028.4531
1      2022-01-01 11:59:59  3728.41  3723.96  3737.00  3721.84   3543.1853
2      2022-01-01 12:59:59  3723.96  3708.21  3728.07  3707.24   6117.2820
3      2022-01-01 13:59:59  3708.20  3735.61  3765.27  3702.84   9353.2056
4      2022-01-01 14:59:59  3735.60  3711.92  3742.67  3708.33   5239.0026
...                    ...      ...      ...      ...      ...         ...
12271  2023-05-27 18:59:59  1826.75  1826.54  1828.00  1824.60   5894.6438
12272  2023-05-27 19:59:59  1826.54  1826.83  1828.36  1822.45   5291.3805
12273  2023-05-27 20:59:59  1826.83  1828.18  1830.40  1824.76   4491.4213
12274  2023-05-27 21:59:59  1828.19  1819.79  1832.00  1812.18  11071.3282
12275  2023-05-27 22:59:59  1819.80  1821.98  1823.19  1818.28   3226.9903

[12276 rows x 6 columns]


In [7]:
df = raw_df.copy()

In [8]:
def check_cross(df, kd_dir):
    up = df['slow_k'] > df['slow_d']
    down = df['slow_k'] < df['slow_d']
    if kd_dir == 'Up':
        return up.diff() & up
    if kd_dir == 'Any':
        return up.diff()
    if kd_dir == 'Down':
        return down.diff() & down

def indicators(df, ema_int):

# ema
    df['ema'] = ta.trend.ema_indicator(df.Close, window=ema_int)

# atr8
    df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)
    
# rsi
    rsi_int = 14
    df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window = rsi_int).rsi()

# kd
    kd_int = 14
    d_int = 3
   
    kd_df = pd.DataFrame()
    kd_df[str(kd_int) + '-Low'] = df['Low'].rolling(kd_int).min()
    kd_df[str(kd_int) + '-High'] = df['High'].rolling(kd_int).max()
    df['slow_k'] = (df['Close'] - kd_df[str(kd_int) + '-Low'])*100/(kd_df[str(kd_int) + '-High'] - kd_df[str(kd_int) + '-Low'])
    df['slow_d'] = df['slow_k'].rolling(d_int).mean()
   

# df['kd_cross'] = check_cross(df, kd_dir)
    
    return df


In [9]:
indicators(df, ema_int)

print(df)

                Close_Time     Open    Close     High      Low      Volume  \
0      2022-01-01 10:59:59  3724.88  3728.32  3738.47  3722.23   5028.4531   
1      2022-01-01 11:59:59  3728.41  3723.96  3737.00  3721.84   3543.1853   
2      2022-01-01 12:59:59  3723.96  3708.21  3728.07  3707.24   6117.2820   
3      2022-01-01 13:59:59  3708.20  3735.61  3765.27  3702.84   9353.2056   
4      2022-01-01 14:59:59  3735.60  3711.92  3742.67  3708.33   5239.0026   
...                    ...      ...      ...      ...      ...         ...   
12271  2023-05-27 18:59:59  1826.75  1826.54  1828.00  1824.60   5894.6438   
12272  2023-05-27 19:59:59  1826.54  1826.83  1828.36  1822.45   5291.3805   
12273  2023-05-27 20:59:59  1826.83  1828.18  1830.40  1824.76   4491.4213   
12274  2023-05-27 21:59:59  1828.19  1819.79  1832.00  1812.18  11071.3282   
12275  2023-05-27 22:59:59  1819.80  1821.98  1823.19  1818.28   3226.9903   

               ema       atr        rsi     slow_k     slow_d  

In [10]:
def conditions(df):

    for index, row in df.iterrows():
        
        df['c1'] = df['Close'].shift(1) < df['ema']
        
        df['c2'] = df['Close'] > df['ema']
        
        df['c3'] = df['Volume'] > df['Volume'].shift(1) * 1.8

        
    # 條件達成
    df['signal'] = False
    df.loc[df.c1 & df.c2 & df.c3, 'signal'] = True


    # 下一根進場
    df['open_entry'] = False
    for i in range(len(df) - 1):
        if df.loc[i, 'signal'] == True:
            df.loc[i + 1, 'open_entry'] = True
    
    return df

In [11]:
conditions(df)

,Close_Time,Open,Close,High,Low,Volume,ema,atr,rsi,slow_k,slow_d,c1,c2,c3,signal,open_entry
0,2022-01-01 10:59:59,3724.88,3728.32,3738.47,3722.23,5028.4531,NaN,0.000000,NaN,NaN,NaN,False,False,False,False,False
1,2022-01-01 11:59:59,3728.41,3723.96,3737.00,3721.84,3543.1853,NaN,0.000000,NaN,NaN,NaN,False,False,False,False,False
2,2022-01-01 12:59:59,3723.96,3708.21,3728.07,3707.24,6117.2820,NaN,0.000000,NaN,NaN,NaN,False,False,False,False,False
3,2022-01-01 13:59:59,3708.20,3735.61,3765.27,3702.84,9353.2056,NaN,0.000000,NaN,NaN,NaN,False,False,False,False,False
4,2022-01-01 14:59:59,3735.60,3711.92,3742.67,3708.33,5239.0026,NaN,0.000000,NaN,NaN,NaN,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12271,2023-05-27 18:59:59,1826.75,1826.54,1828.00,1824.60,5894.6438,1820.448092,5.750097,51.808785,16.840278,18.445977,False,True,False,False,False
12272,2023-05-27 19:59:59,1826.54,1826.83,1828.36,1822.45,5291.3805,1820.698363,5.761518,52.242512,32.040966,20.832089,False,True,False,False,False
12273,2023-05-27 20:59:59,1826.83,1828.18,1830.40,1824.76,4491.4213,1820.991760,5.752838,54.304306,41.916606,30.265950,False,True,False,False,False
12274,2023-05-27 21:59:59,1828.19,1819.79,1832.00,1812.18,11071.3282,1820.944632,6.757636,42.130786,31.787803,35.248458,False,False,True,False,False


In [12]:
entry_df = conditions(df).copy()

in_position = False
stop_loss = np.nan
take_profit = np.nan
close_val = entry_df['Close']
atr_val = entry_df['atr']
open_val = entry_df['Open']


for index, row in entry_df.iterrows():

    if index == 0:
        continue
    
    elif entry_df.at[index, 'open_entry'] == True:
   
        entry_df.at[index, 'entry_p'] = close_val.shift(1).at[index]
        entry_df.at[index, 'stop_loss'] = open_val.shift(1).at[index]
        entry_df.at[index, 'take_profit'] = close_val.shift(1).at[index] + tp_atr * atr_val.shift(1).at[index]
        entry_df.at[index, 'position'] = 'Buy'
        in_position = True
        stop_loss = entry_df.at[index, 'stop_loss']
        take_profit = entry_df.at[index, 'take_profit']
    

    # 吃筍
    #-----------------------------重要-----------------------------
    # 若用 if 寫，則有可能入場馬上吃筍，若用 elif 則一個 iteration 只會執行一次
    elif in_position == True and (entry_df.at[index, sl_det] < stop_loss):
        entry_df.at[index, 'position'] = 'Stop'
        in_position = False
        stop_loss = np.nan
        take_profit = np.nan

    # set take profit
    elif in_position == True and (entry_df.at[index, 'High'] >= take_profit):
        entry_df.at[index, 'position'] = 'Sell'
        in_position = False
        stop_loss = np.nan
        take_profit = np.nan

entry_df.to_csv('entry_df.csv')
        
raw_entry_df = entry_df.copy()

# 過濾有訊號或事件發生的Ｋ線
entry_df = entry_df[(entry_df['open_entry'] == True) |
              (entry_df['signal'] == True) | 
              (entry_df['position'] == 'Buy') |
              (entry_df['position'] == 'Sell') |
              (entry_df['position'] == 'Stop')]


print(entry_df)


                Close_Time     Open    Close     High      Low      Volume  \
78     2022-01-04 16:59:59  3763.88  3770.12  3781.88  3757.58  10982.1712   
79     2022-01-04 17:59:59  3770.12  3766.93  3778.92  3763.59  10956.7217   
112    2022-01-06 02:59:59  3773.00  3747.60  3773.00  3726.40  25781.3228   
218    2022-01-10 12:59:59  3155.04  3175.18  3185.00  3152.00  12759.5370   
219    2022-01-10 13:59:59  3175.18  3174.00  3181.00  3167.40   8635.7041   
...                    ...      ...      ...      ...      ...         ...   
12154  2023-05-22 21:59:59  1810.40  1822.09  1828.27  1808.97  25180.8725   
12155  2023-05-22 22:59:59  1822.09  1819.35  1824.40  1808.54  21301.4725   
12202  2023-05-24 21:59:59  1816.64  1803.04  1818.13  1789.53  39833.3839   
12240  2023-05-26 11:59:59  1801.89  1807.41  1808.52  1801.40   7946.8294   
12241  2023-05-26 12:59:59  1807.41  1808.16  1809.58  1805.28  12227.1474   

               ema        atr        rsi     slow_k     slow_d 

In [13]:
# 部位回測

pos_df = entry_df.copy()
pos_df = pos_df.reset_index(drop = True)
pos_df = pos_df[(pos_df['position'] == 'Buy') |
              (pos_df['position'] == 'Sell') |
              (pos_df['position'] == 'Stop')]

# 一次進場多少單位
pos_size = 1

col = ['Close_Time', 'Open', 'Close', 'High', 'Low', 'ema', 'atr', 'position','entry_p', 'stop_loss', 'take_profit']
pos = pos_df[col]
pos = pos.reset_index(drop = True)


for index, row in pos.iterrows():
    
    current_pos = 0
    
    # 進場
    if pos.at[index, 'position'] == 'Buy':
        pos.at[index, 'size'] = pos_size
        pos.exit_p = np.nan
    
    # 出場
    if pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':
        
        #-----------------------------重要-----------------------------
        # 實戰需即刻出場
        
        # 停利：達成條件時收盤價
        if pos.at[index, 'position'] == 'Sell':
#             pos.at[index, 'exit_p'] = pos.at[index, 'Close']
            for i in range(index -1, -1, -1):
                if pos.at[i, 'position'] == 'Buy':
                    pos.at[index, 'exit_p'] = pos.at[i, 'Close'] + tp_atr * pos.at[i, 'atr']
                break

        # 停損：打到進場停損點（往回跌代，直到最近的'Buy'及其'stop_loss'）
        if pos.at[index, 'position'] == 'Stop':
            for i in range(index -1, -1, -1):
                if pos.at[i, 'position'] == 'Buy':
                    pos.at[index, 'exit_p'] = pos.at[i, 'stop_loss']
                break

        # 計算每次出場部位大小（每次出場皆清倉）
        for i in range(index -1, -1, -1):
            if pos.at[i, 'position'] == 'Buy':
                current_pos += pos.at[i, 'size']
                if i == 0:
                    pos.at[index, 'size'] = -current_pos
                else:
                    continue
            else:
                pos.at[index, 'size'] = -current_pos
                current_pos = 0
                break


# 計算部位價值
for index, row in pos.iterrows():
    if pos.at[index, 'position'] == 'Buy':
        pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'entry_p'], 4)
    elif pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':
        pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'exit_p'], 4)


# 若最後一筆為 Buy，移除該單，迭代驗證
for index, row in pos.iloc[::-1].iterrows():
    if row['position'] == 'Buy':
        pos = pos.drop(index)
    else:
        break
        

# 手續費、滑點、價差
fee = 0.05 / 100
amt_abs_sum = pos.amt.abs().sum()
ttl_fee = amt_abs_sum * fee

# 損益
leverage = 10
ttl_profit = -pos.amt.sum() - ttl_fee


# 計算獲利/虧損次數
agg_amts = []

for i in range(len(pos) - 1, -1, -1):

    if pos.loc[i, 'position'] in ['Stop', 'Sell']:
        
        # look out for the + sign
        total_amt = pos.loc[i, 'amt'] + np.absolute(pos.loc[i, 'amt']) * fee
        
        # iterate backwards from the current row until reaching another 'Stop' or 'Sell'
        # watch out for the + in total_amt += trading_fee
        j = i - 1
        while j >= 0 and pos.loc[j, 'position'] not in ['Stop', 'Sell']:
            total_amt += pos.loc[j, 'amt']
            trading_fee = np.absolute(pos.loc[j, 'amt']) * fee
            total_amt += trading_fee
            j -= 1
        
        # add the aggregated amount to the list
        agg_amts.append(total_amt)

agg_amts.reverse()


# 計算進場最大部位，最大損益
consec_entry = 0
position_amt_sum = 0
max_consec_entry = 0
max_position = 0
max_profit = 0
max_loss = 0

for index, row in pos.iterrows():

    if row['position'] == 'Buy':

        consec_entry += 1
        position_amt_sum += row['amt']

    elif row['position'] in ['Sell', 'Stop']:

        if consec_entry > max_consec_entry:
            max_consec_entry = consec_entry
            max_position = position_amt_sum

        position_amt_sum += row['amt']

        if -position_amt_sum > max_profit:
            max_profit = -position_amt_sum

        if -position_amt_sum < max_loss:
            max_loss = -position_amt_sum

        consec_entry = 0
        position_amt_sum = 0

    else:
        pass


# 最大部位
profit_per = "{:.2f}%".format(ttl_profit / (max_position/leverage) * 100)


# 勝率
wins = 0
loses = 0

for trade in agg_amts:
    if trade < 0:
        wins += 1
    elif trade > 0:
        loses += 1

win_rate = "{:.2f}%".format(wins / (wins + loses) * 100)


# 結果
result = {'Profit': [round(ttl_profit, 2)],
          'Fee': [round(ttl_fee, 2)],
          'Max_Profit': [round(max_profit, 2)],
          'Max_Loss': [round(max_loss, 2)],
          'Max_Entry': [max_consec_entry],
          'Max_Position': [round(max_position, 2)],
          'Profit_%': [profit_per],
          'Win_Rate': [win_rate]}


result_df = pd.DataFrame(result)

print(result_df)
print(pos)
pos.to_csv('pos.csv')



    Profit     Fee  Max_Profit  Max_Loss  Max_Entry  Max_Position Profit_%  \
0  1100.18  215.33      1152.9   -140.08          3       7919.83  138.92%   

  Win_Rate  
0   14.94%  
              Close_Time     Open    Close     High      Low          ema  \
0    2022-01-04 17:59:59  3770.12  3766.93  3778.92  3763.59  3764.249068   
1    2022-01-06 02:59:59  3773.00  3747.60  3773.00  3726.40  3797.292500   
2    2022-01-10 13:59:59  3175.18  3174.00  3181.00  3167.40  3169.393056   
3    2022-01-10 16:59:59  3163.24  3145.27  3169.91  3138.01  3167.927679   
4    2022-01-11 23:59:59  3115.33  3133.56  3158.56  3110.75  3115.989456   
..                   ...      ...      ...      ...      ...          ...   
195  2023-05-17 15:59:59  1818.14  1807.75  1821.22  1803.33  1818.821089   
196  2023-05-19 06:59:59  1814.67  1808.76  1815.73  1807.80  1813.374218   
197  2023-05-22 09:59:59  1798.85  1793.96  1801.72  1792.60  1810.885985   
198  2023-05-22 22:59:59  1822.09  1819.35  182

PermissionError: [Errno 13] Permission denied: 'pos.csv'

In [ ]:
cumulative_values = []
cumulative_sum = 0
for value in agg_amts:
    cumulative_sum -= value
    cumulative_values.append(cumulative_sum)

plt.fill_between(range(len(cumulative_values)), cumulative_values, 0)

plt.xlabel('Index')
plt.ylabel('Cumulative Sum')
plt.title('Cumulative Sum of Values')

plt.show()

In [ ]:
plt_df = raw_entry_df.copy()

plt.figure(figsize=(80, 36))
plt.ylabel('Close')
plt.plot(plt_df.Close, label = 'Close', c ='black')

i = 0
while i < len(plt_df):
    if pd.isna(plt_df.position[i]):
        i += 1
    elif plt_df.position[i] == 'Buy':
        j = i + 1
        while j < len(plt_df) and (pd.isna(plt_df.position[j]) or plt_df.position[j] == 'Buy'):
            j += 1
        if j < len(plt_df) and plt_df.position[j] == 'Sell':
            # Paint all previous 'Buy's green dot
            for k in range(i, j+1):
                plt.plot(k, plt_df.entry_p[k], 'go', label='_', markersize=20)
            i = j
        elif j < len(plt_df) and plt_df.position[j] == 'Stop':
            # Paint all previous 'Buy's red dot
            for k in range(i, j+1):
                plt.plot(k, plt_df.entry_p[k], 'ro', label='_', markersize=20)
            i = j + 1
        else:
            i = j
    else:
        i += 1

plt.legend()
plt.show()


# 之所以出現大量紅點，原因為：連續加倉會調整整個部位停利和停損，因此一些進倉部位會因為調高後的＂整個部位＂停損而出場，但該入場損益依舊為正
# 意思是該部位之停損已經高於許多前面部位之進場點位